In [112]:
#NLP libraries
import findspark
findspark.init()

import re

import pyspark.sql.functions as psf
from pyspark.sql import *
from pyspark.sql.types import *

import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [113]:
# load data 
# keeping only the subtiles
df_text = spark.read.parquet('parquets/films2.parquet').select('tconst','subtitles')
df_ratings = spark.read.parquet('parquets/ratings.parquet')
df_basics = spark.read.parquet('parquets/basics.parquet')

In [114]:
# Joining with ratings to get best and worst movies 
df_tr = df_text.join(other=df_ratings.select('tconst','averageRating'), on='tconst', how='inner')
df_tr = df_tr.join(other=df_basics.select('tconst','genres'), on='tconst', how='inner')

In [115]:
df_tr = df_tr.select("tconst",'averageRating', 'subtitles',psf.explode_outer("genres"))

In [116]:
df_best = df_tr.filter(df_tr['averageRating'] >= 8)
df_worst = df_tr.filter(df_tr['averageRating'] <= 5.2)

# Detokinizing the text

In [117]:
from sacremoses import MosesDetokenizer
def subtitles_to_string(subtitles):
    result = ""
    for subtitle in subtitles:
        test = MosesDetokenizer().detokenize(subtitle)
        result += test + "\n"
    return result
udf_subtitles_to_string = psf.udf(subtitles_to_string, StringType())

In [118]:
df_best = df_best.withColumn("text",udf_subtitles_to_string("subtitles"))
df_best = df_best.drop('subtitles')

In [119]:
df_best.show()

+---------+-------------+---------+--------------------+
|   tconst|averageRating|      col|                text|
+---------+-------------+---------+--------------------+
|tt0032553|          8.5|   Comedy|At the end of the...|
|tt0032553|          8.5|    Drama|At the end of the...|
|tt0032553|          8.5|      War|At the end of the...|
|tt0032599|          8.0|   Comedy|Copy boy!
- Make ...|
|tt0032599|          8.0|    Drama|Copy boy!
- Make ...|
|tt0032599|          8.0|  Romance|Copy boy!
- Make ...|
|tt1280558|          8.2|    Crime|I am retiring tom...|
|tt1280558|          8.2|    Drama|I am retiring tom...|
|tt1280558|          8.2|  Mystery|I am retiring tom...|
|tt0038733|          8.1|   Comedy|This is the unive...|
|tt0038733|          8.1|    Drama|This is the unive...|
|tt0038733|          8.1|  Fantasy|This is the unive...|
|tt0063522|          8.0|    Drama|La la la la-la
La...|
|tt0063522|          8.0|   Horror|La la la la-la
La...|
|tt2488496|          8.0|   Act

In [120]:
df_worst = df_worst.withColumn("text",udf_subtitles_to_string("subtitles"))
df_worst = df_worst.drop('subtitles')

In [121]:
df_worst.show()

+---------+-------------+--------+--------------------+
|   tconst|averageRating|     col|                text|
+---------+-------------+--------+--------------------+
|tt0865554|          5.0|   Crime|What the fuck?
Te...|
|tt0865554|          5.0|   Drama|What the fuck?
Te...|
|tt0865554|          5.0|Thriller|What the fuck?
Te...|
|tt1781775|          4.5|  Horror|[ inaudible male ...|
|tt1781775|          4.5|Thriller|[ inaudible male ...|
|tt0775440|          4.3|  Action|Terminal A of Bos...|
|tt0775440|          4.3|   Drama|Terminal A of Bos...|
|tt0775440|          4.3|  Horror|Terminal A of Bos...|
|tt0962726|          4.7|  Comedy|[ Cheering]
[ Lab...|
|tt0962726|          4.7|   Drama|[ Cheering]
[ Lab...|
|tt0962726|          4.7|  Family|[ Cheering]
[ Lab...|
|tt0106400|          4.9|  Comedy|- I swear, if'n y...|
|tt0106400|          4.9|  Family|- I swear, if'n y...|
|tt0374536|          4.8|  Comedy|It's perfect.
Oh,...|
|tt0374536|          4.8| Fantasy|It's perfect.


# NLP modafucker

In [123]:
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('tagger')

('tagger', <spacy.pipeline.Tagger at 0x1a3545cb70>)

In [147]:
genre = 'Drama'

In [148]:
df_best_pd = df_best.filter(df_best['col'] == genre).toPandas()

In [149]:
df_best_pd.count()

tconst           262
averageRating    262
col              262
text             262
dtype: int64

In [150]:
texts_bests = df_best_pd['text'].values

In [151]:
# remove new lines 
texts_bests = [" ".join(t.split()) for t in texts_bests]

In [ ]:
len(texts_bests)

In [152]:
texts_bests = [text.lower() for text in texts_bests]

### NLP pipe

In [154]:
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS

processed_docs = list()
for doc in nlp.pipe(texts_bests, n_threads=5, batch_size=10):

    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list and keep only words of length 3 or more.
    doc = [token for token in doc if token not in STOPWORDS and len(token) > 2]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)
    
docs = processed_docs ############### variable pour la suite

del processed_docs

In [155]:
# Add bigrams too
from gensim.models.phrases import Phrases

# Add bigrams to docs (only ones that appear 15 times or more).
bigram = Phrases(docs, min_count=4)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/xrubiato/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation

In [156]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]
#MmCorpus.serialize("models/corpus.mm", corpus)

print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(corpus))

Number of unique tokens: 7996
Number of chunks: 262


In [ ]:
seed = 42
# models
from gensim.models import LdaMulticore
params = {'passes': 10, 'random_state': seed}
base_models = dict()

model = LdaMulticore(corpus=corpus,
                     num_topics=3,
                     id2word=dictionary,
                     workers=6,
                     passes=params['passes'],
                     random_state=params['random_state'])

In [157]:
model.show_topics(num_words=10)

[(0,
  '0.010*"THE" + 0.006*"fuck" + 0.006*"YOU" + 0.005*"AND" + 0.004*"George" + 0.003*"Sir" + 0.003*"shit" + 0.003*"Captain" + 0.002*"team" + 0.002*"war"'),
 (1,
  '0.005*"Okay" + 0.004*"fuck" + 0.003*"okay" + 0.003*"Sir" + 0.003*"police" + 0.002*"phone" + 0.002*"Yes_sir" + 0.002*"Mom" + 0.002*"Dad" + 0.002*"dad"'),
 (2,
  '0.005*"Miss" + 0.003*"darling" + 0.003*"Good_night" + 0.002*"gentleman" + 0.002*"Yes_sir" + 0.002*"ought" + 0.002*"war" + 0.002*"party" + 0.002*"Lord" + 0.002*"Goodbye"')]

In [158]:
model.show_topic(1,20)

[('Okay', 0.004794362),
 ('fuck', 0.0044885026),
 ('okay', 0.0031937733),
 ('Sir', 0.0030983803),
 ('police', 0.0030365237),
 ('phone', 0.0024043522),
 ('Yes_sir', 0.0018281002),
 ('Mom', 0.0018107258),
 ('Dad', 0.0017831417),
 ('dad', 0.001771867),
 ('shit', 0.0015477525),
 ('Father', 0.0015239724),
 ('dog', 0.0015131447),
 ('uncle', 0.0014377702),
 ('hospital', 0.0014006392),
 ('Lord', 0.0013574972),
 ('swear', 0.0013457867),
 ('damn', 0.0013217008),
 ('war', 0.0012966959),
 ('power', 0.0012905031)]

In [159]:
sorted(model[corpus[0]],key=lambda x:x[1],reverse=True)

[(1, 0.47612172), (2, 0.38390344), (0, 0.13997483)]

In [ ]:
# plot topics
data =  pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(data)